In [1]:
from src.algo.nn.model import TemporalFusionTransformer

main path: /Users/mengkjin/workspace/learndl
src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']


In [2]:
import torch
import platform

print(f"🖥️  系统: {platform.system()} {platform.machine()}")
print(f"🐍 PyTorch版本: {torch.__version__}")
print(f"🔥 MPS可用: {torch.backends.mps.is_available()}")
print(f"🔥 MPS构建: {torch.backends.mps.is_built()}")

# 如果是Apple Silicon
if platform.machine() == 'arm64':
    print("✅ Apple Silicon Mac - 支持MPS加速")
else:
    print("ℹ️  Intel Mac - 可能需要其他加速方案")

🖥️  系统: Darwin arm64
🐍 PyTorch版本: 2.2.2
🔥 MPS可用: True
🔥 MPS构建: True
✅ Apple Silicon Mac - 支持MPS加速


In [1]:
from src.api import ModelAPI  
ModelAPI.Trainer.available_modules()

main path: /Users/mengkjin/workspace/learndl
src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']


{'simple_lstm': src.algo.nn.model.Recurrent.simple_lstm,
 'gru': src.algo.nn.model.Recurrent.gru,
 'lstm': src.algo.nn.model.Recurrent.lstm,
 'resnet_lstm': src.algo.nn.model.Recurrent.resnet_lstm,
 'resnet_gru': src.algo.nn.model.Recurrent.resnet_gru,
 'transformer': src.algo.nn.model.Recurrent.transformer,
 'tcn': src.algo.nn.model.Recurrent.tcn,
 'rnn_ntask': src.algo.nn.model.Recurrent.rnn_ntask,
 'rnn_general': src.algo.nn.model.Recurrent.rnn_general,
 'gru_dsize': src.algo.nn.model.Recurrent.gru_dsize,
 'patch_tst': src.algo.nn.model.PatchTST.patch_tst,
 'modern_tcn': src.algo.nn.model.ModernTCN.modern_tcn,
 'ts_mixer': src.algo.nn.model.TSMixer.ts_mixer,
 'tra': src.algo.nn.model.TRA.tra,
 'factor_vae': src.algo.nn.model.FactorVAE.FactorVAE,
 'risk_att_gru': src.algo.nn.model.RiskAttGRU.risk_att_gru,
 'ple_gru': src.algo.nn.model.PLE.ple_gru,
 'tft': src.algo.nn.model.TemporalFusionTransformer.TemporalFusionTransformer,
 'lgbm': src.algo.boost.booster.lgbm.Lgbm,
 'ada': src.algo

In [2]:
from src.api import ModelAPI  
app = ModelAPI.short_test(module = 'gru')

AttributeError: type object 'ModelAPI' has no attribute 'short_test'

In [ ]:
from src.api import ModelAPI
ModelAPI.clear_st_models()

In [3]:
from src import INSTANCE_RECORD
INSTANCE_RECORD

InstanceRecord(names=['trainer', 'account'])

In [1]:
from src.api import ModelAPI
ModelAPI.Extractor.update()



model_name is None, update all hidden models
{'name': 'gru_day', 'nums': None, 'submodels': ['best']}
Beware! Should be at server or short_test, but short_test is False now!
try using d:\Coding\learndl\learndl\data\DataSet/day.20240607.pt , success!
Load  2 DataBlocks...... finished! Cost 0.08 secs
Align 2 DataBlocks...... finished! Cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 316/316 [00:00<00:00, 3047.38it/s]

--------------------------------------------------------------------------------


In [1]:
from src.api import ModelAPI

ModelAPI.update_models()

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!
Use device name: NVIDIA GeForce RTX 4090


24-12-15 18:40:32|MOD:display     |: Model Specifics:
24-12-15 18:40:32|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:32 2024!


--------------------------------------------------------------------------------
**************************** RECONSTRUCT TRAIN DATA ****************************
predict is False , Data Processing start!
6 datas : ['y', 'day', '30m', 'style', 'indus', 'week'] , from 20070101 to None
y is up to 20241215121914 already!
day is up to 20241215122636 already!
30m is up to 20241215122719 already!
style is up to 20241215124444 already!
indus is up to 20241215124618 already!
week is up to 20241215124746 already!
Data Processing Finished! Cost 0.00 Seconds
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE MODELS ********************************
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confi

24-12-15 18:40:36|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:36|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:36 2024!
24-12-15 18:40:36|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:36|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:36 2024!
24-12-15 18:40:36|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:36|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:36|MOD:timer       |: Main Process Finished! Cost 3.8 Seconds
24-12-15 18:40:36|MOD:display     |: Model Specifics:
24-12-15 18:40:36|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:36 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRTN_day!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************* SETUP CALLBACKS *******************************
ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
CallbackTimer(verbosity=2) , record time cost of callback hooks
EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 1

24-12-15 18:40:40|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:40|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:40 2024!
24-12-15 18:40:40|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:40|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:40 2024!
24-12-15 18:40:40|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:40|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:40|MOD:timer       |: Main Process Finished! Cost 3.7 Seconds
24-12-15 18:40:40|MOD:display     |: Model Specifics:
24-12-15 18:40:40|MOD:display     |: Start Process [Data] at Sun Dec 15 18:40:40 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
***************************** PARSER TRAINING ARGS *****************************
--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_avg!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************* SETUP CALLBACKS *******************************
ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
CallbackTimer(verbosity=2) , record time cost of callback hooks
EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100,

24-12-15 18:40:44|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-12-15 18:40:44|MOD:display     |: Start Process [Fit] at Sun Dec 15 18:40:44 2024!
24-12-15 18:40:44|MOD:display     |: Finish Process [Fit], Cost 0.0 Secs
24-12-15 18:40:44|MOD:display     |: Start Process [Test] at Sun Dec 15 18:40:44 2024!
24-12-15 18:40:44|MOD:display     |: Testing Mean Score(spearman):
24-12-15 18:40:44|MOD:display     |: Finish Process [Test], Cost 0.0 Secs
24-12-15 18:40:44|MOD:timer       |: Main Process Finished! Cost 3.7 Seconds


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
--------------------------------------------------------------------------------
